In [1]:
import fdapdepy as fdapde
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

# mesh
nodes = np.loadtxt("unit_square/points.txt", dtype=float)
triangles = np.loadtxt("unit_square/elements.txt", dtype=int)
triangles = triangles - 1 # sempre stessa storia
boundary = np.loadtxt("unit_square/boundary.txt", dtype=int)
nodes.shape

mesh = fdapde.domain(nodes, triangles, boundary)
basis = fdapde.functional_basis(mesh, 1)


In [5]:
pde_params = {"diff": 1.}
dir(pde_params)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [6]:
# pde_type = 1 simple laplacian
# -lapl(u) = f
#  u = 0 
# u_ex = sin(pi x) sin(pi y)
u_ex = np.sin(np.pi * mesh.nodes()[:,0]) * np.sin(np.pi*mesh.nodes()[:,1])
f = (1+np.pi*np.pi) * np.sin(np.pi * mesh.nodes()[:,0]) * np.sin(np.pi*mesh.nodes()[:,1])

In [7]:
pde_type = 1
_pde = fdapde.cpp_pde_2_2_1(mesh, int(1), pde_params)

RuntimeError: Unable to cast Python instance of type <class 'float'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)